# Sprint 3 – Webinar 11 (Teórico): KPIs financieros con SQL en **SQL Workbench** (100 min)

**Propósito:** usar SQL para calcular métricas financieras (ingresos, costos, margen y ROI), estructurar y comunicar resultados, y analizar tendencias temporales.

**Herramienta:** **SQL Workbench (en línea)** — https://sql-workbench.com/  
> SQL Workbench corre en tu navegador usando **DuckDB WebAssembly**, así que puedes consultar archivos **CSV/CSV** directamente desde una URL RAW de GitHub. citeturn0search1

**Dataset de práctica (GitHub / Bookstore):** el mismo del cuaderno `Sprint3_Webinar09_Teorico.ipynb` (customers, orders, order_items, products, payments).

**Regla de la clase:** en este webinar **NO usamos CTEs** (`WITH ... AS`). Si necesitas “tablas intermedias”, usamos **tablas temporales** (`CREATE TEMP TABLE ...`) o consultas directas.

---

**Distribución sugerida (100 min):**
- 10 min: Setup + carga del dataset en SQL Workbench  
- 35 min: KPIs financieros (revenue, cost, margin, ROI)  
- 30 min: Reporte (salidas limpias, documentación, recomendaciones)  
- 20 min: Tendencias (series de tiempo + preguntas de negocio)  
- 5 min: Cierre + próximos pasos


## Agenda (100 min)
1) Setup en SQL Workbench + carga del dataset desde GitHub  
2) **Calculating Key Financial Metrics**  
3) **Structuring, Delivering and Communicating Financial Reports**  
4) **Analyzing Trends**  
5) Cierre + tareas


## 0) Setup en SQL Workbench (10 min)

1. Abre **SQL Workbench**: https://sql-workbench.com/ citeturn0search1  
2. Crea un “workspace” nuevo (si aplica) y asegúrate de estar en modo DuckDB (por defecto).  
3. Ejecuta consultas con el botón **Run** (o `Ctrl/⌘ + Enter`).

### Tips de supervivencia
- Si una consulta falla, revisa primero: **coma final, nombre de columna, nombre de tabla**.
- Para ver columnas disponibles usa `DESCRIBE` o `PRAGMA table_info(...)` (abajo tienes ejemplos).


## 1) Cargar el dataset desde GitHub (8–10 min)

En DuckDB (y por tanto en SQL Workbench), puedes consultar **CSV** directo desde una URL RAW. Para no repetir la URL en cada consulta, crearemos **VIEWS**.

> **Importante:** corrige/valida que los nombres de las vistas coinciden con el dataset.

```sql
-- 1) Vistas (tablas lógicas) desde CSV en GitHub
CREATE OR REPLACE VIEW customers AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_customers.parquet';

CREATE OR REPLACE VIEW orders AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_orders.parquet';

CREATE OR REPLACE VIEW order_items AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_order_items.parquet';

CREATE OR REPLACE VIEW products AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_products.parquet';

CREATE OR REPLACE VIEW payments AS
SELECT * FROM 'https://raw.githubusercontent.com/ljpiere/tpdata_python/main/DA/datasets/duckdb/duckdb_bookstore_payments.parquet';
```

Verificación rápida (conteos):

```sql
SELECT 'customers' AS tabla, COUNT(*) AS filas FROM customers
UNION ALL SELECT 'orders',       COUNT(*) FROM orders
UNION ALL SELECT 'order_items',  COUNT(*) FROM order_items
UNION ALL SELECT 'products',     COUNT(*) FROM products
UNION ALL SELECT 'payments',     COUNT(*) FROM payments;
```

### 1.1 Descubrir columnas (muy recomendado)
Antes de construir KPIs, confirma el esquema real:

```sql
DESCRIBE customers;
DESCRIBE orders;
DESCRIBE order_items;
DESCRIBE products;
DESCRIBE payments;

-- Alternativa detallada:
PRAGMA table_info('order_items');
```

### 1.2 Muestras rápidas
```sql
SELECT * FROM customers LIMIT 5;
SELECT * FROM orders LIMIT 5;
SELECT * FROM order_items LIMIT 5;
SELECT * FROM products LIMIT 5;
SELECT * FROM payments LIMIT 5;
```


# 2. Modelo mental: de transacciones a KPIs (5 min)

En este dataset, casi todo KPI financiero se puede construir desde este flujo:

- **orders** = el encabezado (fecha, cliente, estado, etc.)
- **order_items** = el detalle (qué se compró y cuánta cantidad)
- **products** = catálogo (precio, categoría, etc.)
- **payments** = pagos asociados a la orden

Idea clave: **Revenue** suele venir de `order_items × precio` (o de `payments` si quieres medir “dinero efectivamente pagado”).


# 3. Calculating Key Financial Metrics (35 min)

### 3.1 Aggregating Revenue and Cost Data (Revenue & Cost)

**Objetivo:** calcular **revenue** y **cost** por dimensión de negocio (p. ej., categoría / producto).

#### 3.1.1 Revenue (desde order_items + products)
> Ajusta nombres de columnas según tu `DESCRIBE`. En muchos datasets: `order_items.quantity`, `products.price`.

```sql
-- Revenue por categoría (ejemplo)
SELECT
  p.category,
  ROUND(SUM(oi.quantity * COALESCE(oi.unit_price, p.price)), 2) AS revenue
FROM order_items oi
JOIN products p ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY revenue DESC;
```

#### 3.1.2 Cost (desde products.cost)
Como el dataset no siempre trae **costos** unitarios, los simularemos con una tabla temporal `products.cost`.

✅ **Sin CTEs:** creamos una tabla temporal con `VALUES`.

```sql
-- Nota: este dataset ya incluye costo unitario en `products.cost`.
-- Por eso NO necesitamos simular una tabla de costos.

-- Costo por categoría (usando products.cost)
SELECT
  p.category,
  ROUND(SUM(oi.quantity * COALESCE(p.cost, 0)), 2) AS cost
FROM order_items oi
JOIN products p ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY cost DESC;
```

Ahora calculamos **cost** (mapea solo algunos productos; los que no existan quedan NULL → usa `COALESCE` si quieres).

```sql
-- Nota: este dataset ya incluye costo unitario en `products.cost`.
-- Por eso NO necesitamos simular una tabla de costos.

-- Costo por categoría (usando products.cost)
SELECT
  p.category,
  ROUND(SUM(oi.quantity * COALESCE(p.cost, 0)), 2) AS cost
FROM order_items oi
JOIN products p ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY cost DESC;
```


#### Exercise 3.1 (8–10 min)
Escribe una consulta que muestre **revenue** y **cost** por `category` (o por `product_id` si tu tabla no tiene categoría).

Pistas:
- Revenue: `SUM(quantity * price)`
- Cost: `SUM(quantity * cost)` usando `products.cost`
- Redondea con `ROUND(..., 2)`

```sql
-- TU CÓDIGO AQUÍ
```


### 3.2 Calculating Profit and Margin (Gross Profit & Margin)

**Gross Profit:** `revenue - cost`  
**Margin %:** `(revenue - cost) / revenue`

```sql
-- Nota: este dataset ya incluye costo unitario en `products.cost`.
-- Por eso NO necesitamos simular una tabla de costos.

-- Costo por categoría (usando products.cost)
SELECT
  p.category,
  ROUND(SUM(oi.quantity * COALESCE(p.cost, 0)), 2) AS cost
FROM order_items oi
JOIN products p ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY cost DESC;
```


#### Exercise 3.2 (5–7 min)
Calcula el **top 5** de categorías por **gross_profit** (no por revenue).

```sql
-- TU CÓDIGO AQUÍ
```


### 3.3 Measuring ROI by Campaign (ROI)

Si el dataset no trae campañas, simulamos una tabla temporal con **gasto de marketing** por orden.

```sql
DROP TABLE IF EXISTS campaign_spend;

CREATE TEMP TABLE campaign_spend AS
SELECT * FROM (VALUES
  (1, 'Search',  25.00),
  (2, 'Social',  15.00),
  (3, 'Email',    5.00),
  (4, 'Search',  30.00),
  (5, 'Social',  20.00)
) AS t(order_id, campaign, spend_usd);
```

Ahora calculamos ROI por campaña:

- **Revenue de la orden**: suma de `order_items.quantity * price`
- **ROI**: `(revenue - spend) / spend`

```sql
SELECT
  cs.campaign,
  ROUND(SUM(oi.quantity * COALESCE(oi.unit_price, p.price)), 2) AS revenue,
  ROUND(SUM(cs.spend_usd), 2) AS spend,
  ROUND(
    (SUM(oi.quantity * COALESCE(oi.unit_price, p.price)) - SUM(cs.spend_usd))
    / NULLIF(SUM(cs.spend_usd), 0),
    4
  ) AS roi
FROM campaign_spend cs
JOIN order_items oi ON oi.order_id = cs.order_id
JOIN products p     ON p.product_id = oi.product_id
GROUP BY cs.campaign
ORDER BY roi DESC;
```


#### Exercise 3.3 (5–7 min)
Modifica la consulta para que muestre también:
- `profit_after_marketing = revenue - spend`
- Ordena por `profit_after_marketing` desc

```sql
-- TU CÓDIGO AQUÍ
```


### 3.4 Validating and Verifying SQL Results (Sanity checks)

Antes de “publicar” KPIs:
- Verifica duplicados por joins (¿estás multiplicando filas?)
- Revisa nulos y divisiones por cero (`NULLIF`)
- Compara con conteos simples

Ejemplos:

```sql
-- ¿Cuántas órdenes y cuántos items?
SELECT COUNT(*) AS orders FROM orders;
SELECT COUNT(*) AS order_items FROM order_items;

-- ¿Hay órdenes sin items?
SELECT COUNT(*) AS orders_without_items
FROM orders o
LEFT JOIN order_items oi ON oi.order_id = o.order_id
WHERE oi.order_id IS NULL;

-- ¿Hay items con producto inexistente?
SELECT COUNT(*) AS items_without_product
FROM order_items oi
LEFT JOIN products p ON p.product_id = oi.product_id
WHERE p.product_id IS NULL;
```


# 4. Structuring, Delivering and Communicating Financial Reports (30 min)

### 4.1 Organizing SQL Output for Reporting

Un reporte útil suele tener:
- Dimensión (categoría / producto / mes)
- Métrica principal (revenue, margin, ROI)
- Métricas de soporte (número de órdenes, unidades)

Ejemplo: revenue + unidades + #órdenes por categoría.

```sql
SELECT
  p.category,
  COUNT(DISTINCT oi.order_id) AS n_orders,
  SUM(oi.quantity) AS units,
  ROUND(SUM(oi.quantity * COALESCE(oi.unit_price, p.price)), 2) AS revenue
FROM order_items oi
JOIN products p ON p.product_id = oi.product_id
GROUP BY p.category
ORDER BY revenue DESC;
```


#### Exercise 4.1 (8–10 min)
Crea una tabla de reporte por categoría que incluya:
- `revenue`
- `cost` (usando `products.cost`)
- `gross_profit`
- `margin_pct`
- `n_orders`

```sql
-- TU CÓDIGO AQUÍ
```


### 4.2 Documenting SQL Queries with Comments

En trabajo real, el SQL debe ser legible. Usa comentarios para:
- Propósito del query
- Supuestos (p. ej., “revenue calculado desde items”)
- Reglas de filtrado (p. ej., “solo órdenes completadas”)

```sql
-- Reporte financiero por categoría
-- Revenue = SUM(quantity * price)
-- Cost = SUM(quantity * unit_cost) (simulado)
SELECT
  p.category,
  ROUND(SUM(oi.quantity * COALESCE(oi.unit_price, p.price)), 2) AS revenue
FROM order_items oi
JOIN products p ON p.product_id = oi.product_id
GROUP BY p.category;
```


### 4.3–4.5 Resúmenes y recomendaciones (stakeholders)

**Checklist para tu mini-narrativa (3–5 líneas):**
1) Hallazgo (qué pasó)  
2) Evidencia (número / KPI)  
3) Implicación (por qué importa)  
4) Acción recomendada (qué harías)

Ejemplo (plantilla):
- “La categoría **X** aporta **Y%** del revenue pero tiene **margen Z%** (bajo/alto). Recomendamos ___.”

> En clase: convertir 1 tabla de KPIs en 2 recomendaciones: una para negocio (CFO) y otra para marketing.


### 4.6 AI 🤖 – Usar LLMs para crear summary slides (opcional)

Prompt sugerido (copia/pega):
- “Resume estos KPIs en 3 bullets para un CFO y 3 bullets para un gerente de Marketing. Incluye 1 recomendación accionable por rol.”

> Nota: el valor está en tu **tabla limpia** y en que el resumen sea consistente con los números.


# 5. Analyzing Trends (20 min)

### 5.1 Tracking KPIs Over Time (series de tiempo)

Ejemplo: revenue por mes (usando `orders.order_date`).

> Ajusta el nombre de la columna de fecha según `DESCRIBE orders` (`order_date`, `created_at`, etc.).

```sql
SELECT
  date_trunc('month', o.order_date) AS month,
  ROUND(SUM(oi.quantity * COALESCE(oi.unit_price, p.price)), 2) AS revenue
FROM orders o
JOIN order_items oi ON oi.order_id = o.order_id
JOIN products p     ON p.product_id = oi.product_id
GROUP BY month
ORDER BY month;
```


#### Exercise 5.1 (8–10 min)
Calcula **margin_pct por mes** (usa `products.cost`) y responde:
- ¿En qué mes el margen fue más bajo?
- ¿Qué categoría arrastró el margen ese mes? (pista: filtra al mes y agrupa por categoría)

```sql
-- TU CÓDIGO AQUÍ
```


### 5.2 Translating Business Questions into Queries

Ejemplos de preguntas típicas:
- “¿Qué categorías crecieron más en revenue en los últimos 3 meses?”
- “¿Qué productos tienen alto revenue pero bajo margen?”
- “¿Qué campaña tiene ROI alto pero volumen bajo?”

Técnica:
1) Identifica tablas y relaciones  
2) Define la métrica (revenue/margen/ROI)  
3) Define el rango temporal y el nivel de agregación  
4) Valida con sanity checks


#### Exercise 5.2 (5–7 min)
Escribe una consulta para encontrar **top 10 productos** con:
- revenue alto
- margin_pct bajo (ej., < 20%)

```sql
-- TU CÓDIGO AQUÍ
```


### 5.3 AI 🤖 – De pregunta a query (opcional)

Prompt sugerido:
- “Dado este esquema (customers, orders, order_items, products, payments), genera un query para responder: ____ . Evita CTEs y usa solo joins + group by.”

> Revisa siempre nombres de columnas y supuestos.


## Cierre (5 min)

**Takeaways**
- KPIs financieros en SQL = **joins + agregaciones** + cuidado con duplicados.
- Si no hay una tabla “perfecta” (costos / campañas), puedes **simular** con `CREATE TEMP TABLE` para practicar la lógica.
- Documentar el query y entregar una tabla “lista para reporte” vale tanto como el número.

**Tarea sugerida**
1) Ajusta todos los queries a los nombres reales de columnas de tu dataset (según `DESCRIBE`).  
2) Crea un reporte final por categoría con: revenue, cost, gross_profit, margin_pct, n_orders.  
3) Escribe 2 recomendaciones (CFO y Marketing) usando tus números.


## Siguientes pasos
En el próximo sprint/práctico:
- Profundizaremos en validación de datos (calidad, consistencia).
- Practicaremos construcción de dashboards (cuando aplique) y casos reales de preguntas de negocio.
